In [1]:
library(readxl)
Funds <- read_excel("Funds.xlsx")
Bmark <- read_excel("Bmark.xlsx")

Warning message:
"package 'readxl' was built under R version 4.1.2"


In [2]:
download.file(url = "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/Developed_3_Factors_CSV.zip", 
              destfile = 'Developed_3_Factors_CSV.zip')
unzip("Developed_3_Factors_CSV.zip")

In [3]:
Developed_3_Factors <- read.csv('Developed_3_Factors.csv',skip = 6,  nrows = 394)
Developed_3_Factors[-1]<-Developed_3_Factors[-1]*0.01

# Part I Vs. Benchmark Analysis
## I.1. Calculate the annualized ex-post tracking error versus the benchmark
## I.2. Calculate the annualized information ratio relative to the benchmark
## I.3. Calculate the beta relative to the benchmark.

Use base R functions wrapped in a custom function.  

Please calculate the three metrics for each fund’s own individual common return history with the benchmark.  

Create a data frame with the Fund Name, Tracking Error, Info Ratio and Beta as columns. 

Any comments on the metrics for each fund

In [4]:
AnnualizedTrackingError = function(fundname) {
    DT <- merge(na.omit(Funds[,c('Date',fundname)]),Bmark,all.x = TRUE)
    colnames(DT) <- c('Date', 'Fund', 'Benchmark')
    DT$diff<- DT$Fund - DT$Benchmark
    sd(DT$diff)*sqrt(12)
}

In [5]:
AnnualizedInformationRatio = function(fundname) {
    DT <- merge(na.omit(Funds[,c('Date',fundname)]),Bmark,all.x = TRUE)
    colnames(DT) <- c('Date', 'Fund', 'Benchmark')
    DT$diff<- DT$Fund - DT$Benchmark
    SD = sd(DT$diff)*sqrt(12)
    Ex = mean(DT$diff)*12
    Ex / SD
}

In [6]:
Beta = function(fundname){
    DT <- merge(na.omit(Funds[,c('Date',fundname)]),Bmark,all.x = TRUE)
    colnames(DT) <- c('Date', 'Fund', 'Benchmark')
    cor(DT$Fund, DT$Benchmark)*sd(DT$Fund)/sd(DT$Benchmark)
}

In [7]:
Ans1 <- data.frame(matrix(nrow = 3, ncol = 4))
colnames(Ans1) <- c('Fund Name', 'Tracking Error', 'Info Ratio' , 'Beta')
Ans1$'Fund Name'<-colnames(Funds)[2:4]

In [8]:
for (i in (1:3)){
    Ans1$'Tracking Error'[i]<-AnnualizedTrackingError(Ans1$'Fund Name'[i])
    Ans1$'Info Ratio'[i]<-AnnualizedInformationRatio(Ans1$'Fund Name'[i])
    Ans1$'Beta'[i]<-Beta(Ans1$'Fund Name'[i])
}
Ans1

Fund Name,Tracking Error,Info Ratio,Beta
<chr>,<dbl>,<dbl>,<dbl>
Fund1,0.05250632,0.7690834,0.8963913
Fund2,0.04802328,0.9642357,1.0111445
Fund3,0.06673098,0.6425230,1.1127256


$\text{Tracking Error} = \sigma(r_p-r_b)$

$\text{Info Ratio} = \frac{\mu(r_p-r_b)}{\sigma(r_p-r_b)}$

$\text{Beta} = \rho_{p,b} \frac{\sigma(r_p)}{\sigma(r_b)}$

The three metrics measure different perspectives of the fund. Tracking error measures how large the fund deviated from the benchmark, i.e. the active risk, the infomationra ratio measures the excess return per active risk, and the beta measures how the fund moves relative to a benchmark.

From the result, Fund2 has the smallest tracking error or active risk, the highest information ratio, and a close 1 beta. We can conclude Fund2 follows the benchmark closely, and gains most extra return per extra risk when deviate the bechmark. 

Fund3 has the largest tracking error, smallest information ratio, and a >1 beta. The fund gains more when the benchmark is strong and loss more when the benchmark is weak. From the benchmark perspective, it does not perform well. 

Fund1 has a <1 beta, which shows it is not very sensitive to the benchmark.

# Part II Vs. Factor Model

We will be using the Fama-French Developed 3-Factor model as the factor model to compare each of these returns against.  Run a basic linear regression for each fund versus this 3-factor model and be sure to account for the risk-free rate (included with the French data) on the left-hand side of the regression.  Create a data frame with the following columns: Fund Name, Alpha (annualized), Alpha t-Stat, Developed Market Beta, Developed Market t-Stat, Developed Size Beta, Developed Size t-Stat, Developed Value Beta, Developed Value t-Stat, Adj. R2. Any comments on any of the regression results?

In [9]:
Developed_3_Factors$YYMM <- format(as.POSIXct(paste0(Developed_3_Factors$X, "01"), format = "%Y%m%d", tz = "UTC"),'%Y%m')
Funds$YYMM<-format(Funds$Date,'%Y%m')

In [10]:
LRSummary = function(fundname){
    DT <- merge(na.omit(Funds[,c('YYMM',fundname)]),
                Developed_3_Factors[,colnames(Developed_3_Factors)[-1]],all.x = TRUE)
    colnames(DT)[2]<-'fund'
    Y <- DT$fund - DT$RF
    Market<- DT$Mkt.RF
    Size <-DT$SMB
    Value<-DT$HML
    LM <- lm(Y~Market+Size+Value)
    Summary <- summary(LM)
    Coefficients <- Summary$coefficients
    c(Coefficients['(Intercept)','Estimate']*12, Coefficients['(Intercept)','t value'],
      Coefficients['Market','Estimate'], Coefficients['Market','t value'],
      Coefficients['Size','Estimate'], Coefficients['Size','t value'],
      Coefficients['Value','Estimate'], Coefficients['Value','t value'],
      Summary$adj.r.squared
     )
}

In [11]:
Ans2 <- data.frame(matrix(nrow = 3, ncol = 10))
colnames(Ans2) <- c('Fund Name', 'Alpha (annualized)', 'Alpha t-Stat' , 
                   'Developed Market Beta','Developed Market t-Stat', 
                   'Developed Size Beta','Developed Size t-Stat',
                   'Developed Value Beta','Developed Value t-Stat',
                   'Adj. R2'
                  )
Ans2$'Fund Name'<-colnames(Funds)[2:4]

In [12]:
for (i in (1:3)){
    Ans2[i,2:ncol(Ans2)] <- LRSummary(Ans2$'Fund Name'[i])
}
Ans2

Fund Name,Alpha (annualized),Alpha t-Stat,Developed Market Beta,Developed Market t-Stat,Developed Size Beta,Developed Size t-Stat,Developed Value Beta,Developed Value t-Stat,Adj. R2
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Fund1,0.01481489,1.461491,0.5705201,30.42173,-0.12103405,-3.044214,0.26705126,8.011594,0.7987610
Fund2,0.02151705,2.300831,0.6400152,37.08760,0.05939853,1.562890,0.15254096,5.029381,0.8573744
Fund3,0.02243838,1.540541,0.6816240,26.09011,0.09064105,1.646070,-0.08324953,-1.774606,0.7756220


From the table above, we can conclude that:

Fund1 is significant on all three factors, but its alpha is not significant. The three factor model can explain 79.9% of the funds variance. It has positive sensitivity to market return and value return, but negative sensitivity to size return.

Fund2 is significant on alpha,  market return, and value return, but beta on size return is not significant. The three factor model can explain 85.7% of the funds variance. It can generate extra 0.18% annual return compare to the market. It has positive sensitivity to market return and value return.

Fund3 is only significant on market. The three factor model can explain 77.6% of the funds variance. 